# Youtube spam detection

Download [link](http://www.dt.fee.unicamp.br/~tiago//youtubespamcollection/)

In [1]:
from fastai2.text.all import *
from lazylabel.text.all import *

In [2]:
source = Path('/home/lgvaz/.data/youtube_spam/')

In [3]:
fns = get_files(source, extensions='.csv')
dfs = [pd.read_csv(fn) for fn in fns]

In [4]:
df = pd.concat(dfs)
print(df.size); df.head()

9780


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free Leads on Auto Pilot &amp; You Can Too! http://www.MyLeaderGate.com/moretraffic﻿,1
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-shop/6174122/fan-army-face-off-round-3 Vote for SONES please....we're against vips....please help us.. &gt;.&lt;﻿,1
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,"Hey guys! Please join me in my fight to help abused/mistreated animals! All fund will go to helping pay for vet bills/and or helping them find homes! I will place an extra emphasis on helping disabled animals, ones otherwise would just be put to sleep by other animal organizations. Donate please. http://www.gofundme.com/Angels-n-Wingz﻿",1
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the song﻿,1
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http://believemefilm.com?hlr=h2hQBUVB﻿,1


In [5]:
df['original'] = df['CONTENT']

In [6]:
cder,tkzer,nmzer = mk_transform(ColReader('original')),Tokenizer.from_df('CONTENT'),Numericalize()

In [7]:
ABSTAIN,SPAM,HAM = 'abstain','spam','ham'
vocab = [ABSTAIN,SPAM,HAM]

In [8]:
labeller = Labeller()
@labeller(tkzer)
def check(x): return SPAM if 'check' in x else ABSTAIN
@labeller(cder)
def check_out(x): return SPAM if 'check out' in x.lower() else ABSTAIN

In [9]:
tls = TfmdLists(df, [cder,tkzer,nmzer])
tasks = tasks_labels(labeller, tls, vocab)

In [10]:
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [11]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.42% (392),1
check,23.75% (456),1


In [12]:
def get_sentences(tls, idxs):
    sentences = {}
    for i in idxs:
        tks = tls.decode(tls[i])
        sentences[i] = (' '.join(decode_spec_tokens(tks.split(' '))))
    return pd.DataFrame(sentences.values(), sentences.keys())

In [13]:
idxs = labeller.find(dls.train, vocab, ['check', 'check_out'], [SPAM, ABSTAIN])

In [14]:
get_sentences(tls, idxs).sample(10)

,0
1055,"xxbos Hey guys can you check my youtube channel i know you hate comments like this one but i promise if you check my videos it will be xxunk i do Xxunk xxunk , xxunk Xxunk Xxunk and Xxunk please guys can you check them out and thanks have a good day !!!!!!! ﻿﻿﻿﻿﻿﻿﻿"
1419,xxbos Hi everyone . We are a xxunk and we are xxunk to record xxunk and put them on youtube . If any of you could check it out and like / comment it would mean so much to us because we love doing this . We may not have the best recording equipment but if you listen to our lyrics and xxunk i think you&'ll like it . If you do then please subscribe and share because we love making these videos and we want you to like them as much as possible so feel free to comment and give us xxunk ! Thank you !
97,xxbos Great video by a great artist in Katy Perry ! Xxunk her xxunk ! Check out our channel for no xxunk xxunk xxunk reviews and xxunk as well as an iphone 6 and 6 Plus xxunk and xxunk
724,xxbos watch?v = xxunk Check this out .﻿
708,"xxbos Hey Music Fans i really appreciate all of you who take time to read this , and check my music out ! I&'m just a 15 year old boy DREAMING of being a successful MUSICIAN in the music world . i do lots of covers , and piano covers . But i do nt have money to advertise . a simple thumbs up to my comment , a comment on my videos or a SUBSCRIPTION would be a step forward ! It will only be a few seconds of your life that u won&'t regret !!! Thank u to all the people who just give me a chance it means a lot ! :)"
1051,xxbos Hey come check us out were new on youtube let us know what you think and do n't forget to subscribe xxunk
159,"xxbos a friend of mine has xxunk a big dick xxunk . He had a small dick ( 4 xxunk ) and he did some xxunk about this xxunk . Xxunk the xxunk , he found out the secret xxunk of xxunk xxunk . He xxunk what he had learned and now he has a 7 xxunk dick . He was absolutely xxunk by his xxunk . Of xxunk , it took a few months . Xxunk , he has xxunk a book about this xxunk in order to help guys . He xxunk me to xxunk his book :) So , guys if you are xxunk and for more xxunk check this out https : / / xxunk / b / xxunk . He is xxunk it for xxunk xxunk 1 . Thank you ; ) ﻿"
1012,xxbos Hi guys my name is Xxunk and i do XXUNK football videos i have xxunk subscribers and i think you guys would like my content so come check it out and if you do subscribe!﻿
751,"xxbos Came here to check the views , xxunk"
110,"xxbos Hey guys ! I 've made a amazing Xxunk xxunk all the things you xxunk , your xxunk is the most xxunk and remember all the xxunk in the world ca n’t xxunk the xxunk of a smile . If you 're a big fan of t - shirts and want to gets more xxunk , it 's perfect for you . Check this out and buy it at www xxunkxxunkxxunk /// xxunkxxunkxxunk =)=)=) ))) thanksthanksthanks youyouyou guysguysguys sososo muchmuchmuch 333 !!! ﻿﻿﻿"


In [15]:
@labeller(cder)
def regex_check_out(x):
    return SPAM if re.search(r'check.*out', x, flags=re.I) else ABSTAIN

In [16]:
@labeller(cder)
def regex_check(x):
    return SPAM if re.search(r'(check\s(my|this))|check\s(\w+)?\s?out', x, flags=re.I) else ABSTAIN

In [17]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [18]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.68% (397),1
regex_check_out,22.66% (435),1
regex_check,23.28% (447),1
check,24.11% (463),1


In [19]:
idxs = labeller.find(dls.train, vocab, ['regex_check', 'regex_check_out'], [SPAM, ABSTAIN])
print(len(idxs))
get_sentences(tls, idxs).sample(10)

17


,0
764,"xxbos if you like xxunk talent , xxunk lyrics , xxunk real hip hop Everyone check my newest sound Xxunk x - Got the Xxunk ( prod by . Xxunk the Model Xxunk ) COMMENT TELL ME WHAT YOU THINK DO nt BE XXUNK !!!! ---- 1111 //// 7777 Xxunk"
25,xxbos Hi Guys ! check this awesome EDM & & House mix :) thanks a lot .. https : / / soundcloud.com / xxunk / edm - house - xxunk
729,xxbos and u xxunk check my channel and tell me what i should do xxunk
812,"xxbos Check my channel , xxunk"
34,"xxbos Hello ! I 'm Xxunk , I 'm a singer from Xxunk ! i was part of a boy - girl band named xxunk , and I 'm now singing on my own xxunk i wanted to play my own pop and pop - rock songs . It would mean a lot if you could have a look at my channel to check my music and watch my new video ! ! and if u like , subscribe to it ! XOXO THANKS ! ! XXUNK : if you like a lot my channel , you can share it with your friends ! ! Haha ! ! LOL XXUNK"
880,xxbos CHECK MY CHANNEL FOR MY NEW SONG ' xxunk ' ! ! YOU 'll LOVE XXUNK
91,xxbos Can you check my videos please ? Do n't hate me :( Give me one xxunk
804,xxbos Check my channel﻿
52,"xxbos Hey guys go to check my video name "" xxunk my xxunk"
1820,"xxbos Need money ? check my channel and subscribe , soon will post how to get it ) ﻿"


In [20]:
idxs = labeller.find(dls.train, vocab, ['regex_check'], [SPAM])
get_sentences(tls, idxs).sample(10)

,0
681,xxbos Hi . Check out and share our songs .
1858,xxbos Check out this video on youtube : xxunk
1392,"xxbos i hope everyone is in good xxunk I&'m a hard working student who & 's also a passionate singer i look xxunk to the day when i can make my own music to share But for now I&'ve just been doing covers . Check out my channel , I&'ve done Covers of Miley Cyrus , Imagine Dragons , Xxunk Xxunk Xxunk , Xxunk , Xxunk , Xxunk and xxunk others . Subscribe only if you want to . My goal isn&'t to become famous but to xxunk XXUNK this isn&'t spamming , everyone has a right to freedom of speech . Thanks"
1096,xxbos Check out this playlist on youtube:﻿
753,"xxbos Check out my dubstep song "" xxunk "" , made with Xxunk Xxunk . i really took time in it . / watch?v = xxunk"
52,"xxbos Hey guys go to check my video name "" xxunk my xxunk"
1339,"xxbos This great Xxunk will happen soon . , 0 \n xxunk , xxunk xxunk - 09 - xxunk 000 ,,, xxunkxxunkxxunk <<< xxunkxxunkxxunk \n\n\n xxunkxxunkxxunk ,,, ggg xxunkxxunkxxunk --- 090909 --- xxunkxxunkxxunk 333 000 ,,, nicenicenice xxunkxxunkxxunk \n\n\n xxunkxxunkxxunk ,,, xxunkxxunkxxunk ... xxunkxxunkxxunk ... xxunkxxunkxxunk --- 090909 --- xxunkxxunkxxunk 444 000 ,,, 600600600 mmm viewsviewsviews ... xxunkxxunkxxunk \n\n\n xxunkxxunkxxunk ,,, xxunkxxunkxxunk Xxunk xxunk - 09 - xxunk 333 000 ,,, fuckfuckfuck xxunkxxunkxxunk \n\n\n xxunkxxunkxxunk --- xxunkxxunkxxunk --- eee ,,, xxunkxxunkx..."
1201,"xxbos You guys should check out this EXTRAORDINARY website called FIREPA.COM . You can make money online and start working from home today as i am ! i am making over $ 3 , 000 +++ perperper monthmonthmonth atatat FIREPA.COM ! Visit FIREPA.COM and check it out ! Lake . Xxunk . Xxunk . Xxunk . Chillpal . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk"
976,xxbos Hey guys ! Check this out : Xxunk - Do n't be xxunk ' my xxunk ! i think that they deserve much more xxunk than they xxunk
1889,xxbos Yo like what up this song is xxunk xxunk xxunk xxunk the see xxunk xxunk or i xxunk up in a new Xxunk check out xxunk rap i was xxunk in it see my nice xxunk xxunk things xxunk for xxunk xxunk xxunk xxunk i was at my xxunk house doing it we just had the rap going and we were doing fun things ﻿


In [21]:
labeller.remove('check_out')
labeller.remove('regex_check_out')

In [22]:
from textblob import TextBlob

In [23]:
@labeller(cder, TextBlob)
def tb_polarity(x, tb):
    return HAM if tb.polarity > 0.9 else ABSTAIN
@labeller(cder, TextBlob)
def tb_sentiment(x, tb):
    return HAM if tb.subjectivity >= 0.5 else ABSTAIN

In [24]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [25]:
labeller.summary(dls.train)

,Coverage,Polarity
regex_check,23.23% (446),1
tb_polarity,4.79% (92),1
tb_sentiment,37.97% (729),1
check,24.06% (462),1


In [26]:
idxs = labeller.find(dls.train, vocab, lfs=['tb_sentiment'], lbls=[HAM])
get_sentences(tls, idxs).sample(10)

,0
224,xxbos want to win xxunk the xxunk - xxunk ? check my channel : ) ﻿
622,xxbos i love this song
1595,xxbos cool cool cool cool cool cool cool ﻿ ﻿ ﻿ ﻿ ﻿ ﻿ ﻿
1199,"xxbos You guys should check out this EXTRAORDINARY website called FIREPA.COM . You can make money online and start working from home today as i am ! i am making over $ 3 , 000 +++ perperper monthmonthmonth atatat FIREPA.COM ! Visit FIREPA.COM and check it out ! Lake . Chillpal . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk"
672,xxbos Please .. Check my channel out : ) i subscribe back .. ; )
361,xxbos Why so many xxunk ?????? 666666 !!!!!! xxunkxxunkxxunkxxunkxxunkxxunk ﻿﻿﻿﻿﻿﻿
1383,xxbos Eminem - Love the way you lie ♥ ♥ ♥ ﻿
1211,"xxbos You guys should check out this EXTRAORDINARY website called MONEYGQ.COM . You can make money online and start working from home today as i am ! i am making over $ 3 , 000 +++ perperper monthmonthmonth atatat MONEYGQ.COM ! Visit MONEYGQ.COM and check it out ! Lake . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk . Xxunk"
1665,xxbos everyday I&'m xxunk
375,xxbos Nice love i tttttttt xxunkxxunkxxunkxxunkxxunkxxunkxxunkxxunk xxunkxxunkxxunkxxunkxxunkxxunkxxunkxxunk xxunkxxunkxxunkxxunkxxunkxxunkxxunkxxunk ﻿﻿﻿﻿﻿﻿﻿﻿


In [27]:
def keyword_match(ws, label=SPAM):
    ws = L(ws)
    def _inner(x):
        for w in ws:
            if w in x: return label
        return ABSTAIN
    _inner.__name__ = f'kw_{ws[0]}'
    labeller.register_func(cder, _inner)

In [28]:
keyword_match('my')
keyword_match('subscribe')
keyword_match(['http', 'https'])
keyword_match(['please', 'plz', 'pliz'])
keyword_match(['song'], label=HAM)

In [29]:
@labeller(tkzer)
def short_comment(x): return HAM if len(x) < 6 else ABSTAIN

In [30]:
tasks = tasks_labels(labeller, tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [31]:
labeller.summary(dls.train)

,Coverage,Polarity
regex_check,23.23% (446),1
tb_polarity,4.74% (91),1
tb_sentiment,38.07% (731),1
kw_my,16.35% (314),1
kw_subscribe,8.44% (162),1
kw_http,10.05% (193),1
kw_please,6.04% (116),1
kw_song,14.17% (272),1
check,24.01% (461),1
short_comment,11.82% (227),1


In [32]:
import spacy
nlp = spacy.load('en_core_web_sm')
def spacy_doc(x): return nlp(str(x))

In [33]:
@labeller(cder, spacy_doc)
def has_person(x, sdoc):
    if len(sdoc) < 20 and any([ent.label_=='PERSON' for ent in sdoc.ents]):
        return HAM
    return ABSTAIN

In [34]:
splits = RandomSplitter()(df)
tasks = tasks_labels(labeller, tls, vocab)
lbls =  TfmdLists(df, [ColReader('CLASS'), {1:SPAM, 0:HAM}.get, Categorize(vocab)])
dset = Datasets(tls=[tasks, lbls], tfms=[], splits=splits)
dls = dset.dataloaders(n_workers=0)

In [35]:
labeller.summary(dls.train)

,Coverage,Polarity,Accuracy,Correct,Incorrect
regex_check,23.33% (448),1,1.000000,448,0
tb_polarity,4.69% (90),1,0.822222,74,16
tb_sentiment,37.97% (729),1,0.573388,418,311
kw_my,16.35% (314),1,0.875796,275,39
kw_subscribe,8.39% (161),1,0.987578,159,2
kw_http,10.16% (195),1,0.948718,185,10
kw_please,6.09% (117),1,1.000000,117,0
kw_song,14.22% (273),1,0.732601,200,73
has_person,6.35% (122),1,0.606557,74,48
check,24.17% (464),1,0.971983,451,13


In [53]:
idxs = labeller.find(dls.train, vocab, ['has_person'], [HAM])

In [59]:
text = df.iloc[1037].original
sdoc = spacy_doc(text)

In [54]:
get_sentences(tls, idxs).sample(10)

,0
1357,xxbos Like & & Subscribe / xxunk
860,xxbos http : / / www xxunkxxunkxxunk /// xxunkxxunkxxunk /// xxunkxxunkxxunk === xxunkxxunkxxunk
1655,xxbos Xxunk Xxunk little kid xxunk xxunk
1386,xxbos Is that girl is Megan fox ﻿
1688,xxbos i could finally do the xxunk but i started yesterday XD﻿
1037,xxbos Check out my Music Videos ! Xxunk - u LA LA Remix xxunk / xxunk
1159,xxbos Check Out The New Hot Video By Dante b Called Riled Up
306,xxbos Katy Perry - Roar ( official ) : http : / / youtu.be / cevxzvsjlk8﻿
269,"xxbos When i hear Katy singing this , i xxunk . The song has got xxunk"
1706,xxbos Who xxunk is Lauren Bennett .. ﻿
